In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.train.estimator import Estimator
import json

In [2]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="39288a38-ff19-432c-8011-1cd9d0dff445")
ws = Workspace(subscription_id="793146d9-d4dc-4a73-9728-76c4ffd0cc0d", resource_group="rg_dynamics_test", workspace_name="resdynml1test", auth=interactive_auth)

In [ ]:
# load experiment cfg
with open("experiment_cfg.json", "r") as cfg_file:
    cfg = json.load(cfg_file)

## Training Scripts

In [14]:
%%writefile ./src/train.py

from azureml.core import Run

import os
import joblib
import numpy as np
from argparse import ArgumentParser
from sklearn.metrics import recall_score, precision_score, hamming_loss, zero_one_loss, mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense

run = Run.get_context()

parser = ArgumentParser()
parser.add_argument('--input', dest='preprocessed_data')
args = parser.parse_args()

############################################################

# load data
if args.preprocessed_data:
    train_data = pd.read_csv(args.preprocessed_data + '/train_data.csv', sep=';', header=0)
    test_data = pd.read_csv(args.preprocessed_data + '/test_data.csv', sep=';', header=0)
else:
    train_data = run.input_datasets['train_data'].to_pandas_dataframe()
    test_data = run.input_datasets['test_data'].to_pandas_dataframe()
    
############################################################

# split train/test and feat/target
X_train = train_data[[ col for col in train_data.columns if col.startswith('feat')]].values
y_train = train_data[[ col for col in train_data.columns if col.startswith('target')]].drop(['target_0'], axis=1).values
X_test = test_data[[col for col in test_data.columns if col.startswith('feat')]].values
y_test = test_data[[ col for col in test_data.columns if col.startswith('target')]].drop(['target_0'], axis=1).values

############################################################

# create model
inputs = Input(shape=(X_train.shape[1],))

x = Dense(128, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)

outputs = Dense(y_train.shape[1], activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
run.log('Model Summary', model.summary())

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
# tf.keras.metrics.Recall(top_k=y_train.shape[1])

# train classifier
history = model.fit(X_train, y_train, batch_size=64, epochs=10)

##############################################################

# evaluate test data
y_pred = model.predict(X_test).round()
run.log_table(
    'test_evaluation_classification',
    {
        'precision_macro': [precision_score(y_test, y_pred, average='macro')],
        'precision_samples': [precision_score(y_test, y_pred, average='samples')],
        'recall_macro': [recall_score(y_test, y_pred, average='macro')],
        'recall_samples': [recall_score(y_test, y_pred, average='samples')],
        'hamming_loss': [hamming_loss(y_test, y_pred)],
        'zero_one_loss': [zero_one_loss(y_test, y_pred)]
    }
)

# evaluate train data
y_pred = model.predict(X_train).round()
run.log_table(
    'train_evaluation_classification',
    {
        'precision_macro_train': [precision_score(y_train, y_pred, average='macro')],
        'precision_samples_train': [precision_score(y_train, y_pred, average='samples')],
        'recall_macro_train': [recall_score(y_train, y_pred, average='macro')],
        'recall_samples_train': [recall_score(y_train, y_pred, average='samples')],
        'hamming_loss_train': [hamming_loss(y_train, y_pred)],
        'zero_one_loss_train': [zero_one_loss(y_train, y_pred)]
    }
)

# save model
os.makedirs('outputs', exist_ok=True)
#joblib.dump(value=model, filename='outputs/model.pkl')
model.save('outputs/model.pkl')

############################################################

# train regressor
X_train = train_data[[ col for col in train_data.columns if col.startswith('feat')]].values
y_train = train_data[[ col for col in train_data.columns if col.startswith('target')]][['target_0']].values
X_test = test_data[[col for col in test_data.columns if col.startswith('feat')]].values
y_test = test_data[[ col for col in test_data.columns if col.startswith('target')]][['target_0']].values
y_test = y_test.values[:,0]

############################################################

# create model
inputs = Input(shape=(X_train.shape[1],))

x = Dense(128, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)

outputs = Dense(y_train.shape[1], activation='linear')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
run.log('Model Summary', model.summary())

model.compile(loss='root_mean_squared_error', optimizer='adam', metrics=["mae"])
# tf.keras.metrics.Recall(top_k=y_train.shape[1])

# train classifier
history = model.fit(X_train, y_train, batch_size=64, epochs=10)

############################################################

# evaluate test data
y_pred = model_regressor.predict(X_test)
run.log_table(
    'test_evaluation_regression',
    {
        'mae': [mean_absolute_error(y_test, y_pred)],
        'mse': [mean_squared_error(y_test, y_pred)],
        'r2': [r2_score(y_test, y_pred)]
    }
)

# evaluate train data
y_pred = model_regressor.predict(X_train)
run.log_table(
    'train_evaluation_regression',
    {
        'mae_train': [mean_absolute_error(y_train, y_pred)],
        'mse_train': [mean_squared_error(y_train, y_pred)],
        'r2_train': [r2_score(y_train, y_train)]
    }
)

############################################################

# save regressor model
# joblib.dump(value=model_regressor, filename='outputs/model_regressor.pkl')
model.save('outputs/model_regressor.pkl')

run.complete()

Overwriting ./src/train.py


## Create Estimator

In [15]:
# est = TensorFlow(entry_script='train.py', source_directory='src',
#              inputs=[    ws.datasets['DummyPredictionTrain'].as_named_input('train_data'), 
#                          ws.datasets['DummyPredictionTest'].as_named_input('test_data')],
#              compute_target='local', pip_packages=['pyarrow==0.12.0', 'joblib'],
#                  framework_version='2.0')#, environment_definition=ws.environments[env_name])

In [16]:
est = Estimator(entry_script='train.py', source_directory='src',
                inputs=[ws.datasets[cfg['train_dataset']].as_named_input('train_data'), 
                        ws.datasets[cfg['test_dataset']].as_named_input('test_data')   ],
                compute_target='local', environment_definition=ws.environments[cfg['env_name']])

## Run Experiment

In [17]:
exp = Experiment(ws, cfg['experiment_name'])
run = exp.submit(est)
run.wait_for_completion(show_output=True)

RunId: KerasPrediction_1592470984_42b9f8cc
Web View: https://ml.azure.com/experiments/KerasPrediction/runs/KerasPrediction_1592470984_42b9f8cc?wsid=/subscriptions/793146d9-d4dc-4a73-9728-76c4ffd0cc0d/resourcegroups/rg_dynamics_test/workspaces/resdynml1test

Streaming azureml-logs/70_driver_log.txt

Entering context manager injector. Current time:2020-06-18T09:03:07.187771
Starting the daemon thread to refresh tokens in background for process with pid = 31195
Entering Run History Context Manager.
Preparing to call script [ train.py ] with arguments: []
After variable expansion, calling script [ train.py ] with arguments: []

2020-06-18 09:06:32.318121: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-06-18 09:06:32.324187: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2095195000 Hz
2020-06-18 09:06:32.325193: I tensorflow/compiler/xla/service/service.c

12209/12209 [==============================] - 1s 58us/sample - loss: 0.0051 - accuracy: 0.9988
Epoch 8/10

12209/12209 [==============================] - 1s 58us/sample - loss: 0.0050 - accuracy: 0.9988
Epoch 9/10

12209/12209 [==============================] - 1s 57us/sample - loss: 0.0050 - accuracy: 0.9988
Epoch 10/10

12209/12209 [==============================] - 1s 60us/sample - loss: 0.0050 - accuracy: 0.9988
2020-06-18 09:06:40.732965: W tensorflow/python/util/util.cc:299] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
WARNING - From /home/azureuser/.azureml/envs/azureml_11a74f652b38787cd8ba913cc91d6de1/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a fut

{'runId': 'KerasPrediction_1592470984_42b9f8cc',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-06-18T09:03:06.417289Z',
 'endTimeUtc': '2020-06-18T09:06:44.465605Z',
 'warnings': [{'source': 'RunHistoryService',
   'message': 'Failed to post metric due to validation failure'}],
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '01764806-3d89-438d-b819-e6e1413b8c22'},
 'inputDatasets': [{'dataset': {'id': 'b217579b-16ba-4c90-9e95-1776ba592742'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'test_data', 'mechanism': 'Direct'}}, {'dataset': {'id': '723a8324-8b50-48ba-9398-53c659959f56'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'train_data', 'mechanism': 'Direct'}}],
 'runDefinition': {'script': 'train.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'test_data': {'dataLo

## Register Models

In [ ]:
# register trained model
run.register_model(cfg['TrainedClassifier'], 'outputs/model.pkl')
run.register_model(cfg['TrainedRegressor'], 'outputs/model_regressor.pkl')